In [16]:
# Training text is Thus Spoke Zarathustra concatenated with Dostoyevsky's 'The Idiot'
# Guided by Andrej Karpathy: https://www.youtube.com/watch?v=kCc8FmEb1nY

In [2]:
import tensorflow as tf

# import numpy as np
from tokenizer import TextProcessor

tp = TextProcessor("TSZ_input.txt")
data = tp.text

data_enc = tp.map_to_int(data)
data_enc = tf.convert_to_tensor(data_enc)

train_split = int(len(data) * 0.9)
xtrain = data_enc[:train_split]
val_data = data_enc[train_split:]

- The `tensor([76 62 63 1 75 63 60 80 83])` actually contains 7 different training examples in it; lets assume that these char to int 'embeddings' represent the word 'Nietzsche' the different examples would be:
  - `[76]` (aka. `['N']`) -> is likely proceeded by a `[62]`(or `['i']`)
  - `[76 62]` (aka. `['Ni']`) -> is likely proceeded by a `[63]` (or `['e']`)
  - ...
  - `[76 62 63  1 75 63 60 80]` (aka. `['Nietzsch']`) -> is likely proceeded by `[83]` (or `['e']`)
    <br>
    <br>
- transformer will never receive more than chunk_size tokens/inputs at a time
  The below operation is not only done for efficiency but also so the transformer is 'used' to seeing context of size 1..chunk_size


In [3]:
chunk_size = 8
x = xtrain[:chunk_size]
y = xtrain[1 : chunk_size + 1]

for k in range(chunk_size):
    context = x[: k + 1]
    target = y[k]
    # print(f'context: {context} target: {target}')
# print(tf.__version__)
# tp.stoi['T']

In [4]:
tf.random.set_seed(44)
batch_size = 32
# Chunksize = context length, number of tokens to be considered
# chunk_size = 8

# get training batch
# tf.function because the global seed is set and operation seeds are not set
# @tf.function
# However, adding the tf.function decorator causes the following error:
# OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.


def get_batch(dat):
    dat = xtrain if dat == "train" else val_data
    randint = tf.random.uniform(
        shape=(batch_size,), maxval=len(dat) - chunk_size, dtype=tf.int64
    )
    x = tf.stack([dat[i : i + chunk_size] for i in randint])
    y = tf.stack([dat[i + 1 : i + chunk_size + 1] for i in randint])
    # print(f"randint: {randint}, randint.shape: {randint.shape}")
    return x, y


xs, ys = get_batch("train")

"""
x=[1,60,69..10]
y = [60,69,..1]
input x[0,0:2]= [1,60] output is y[2] = 69
"""

for b in range(batch_size):
    for k in range(chunk_size):
        context = xs[b, : k + 1]
        target = ys[b, k]
        # print(f'context: {context} target: {target}')

xs.shape, ys.shape, type(xs), type(ys)

(TensorShape([32, 8]),
 TensorShape([32, 8]),
 tensorflow.python.framework.ops.EagerTensor,
 tensorflow.python.framework.ops.EagerTensor)

Stanford n-gram LM [source](https://web.stanford.edu/~jurafsky/slp3/3.pdf)


In [48]:
from tensorflow import keras
from typing import Tuple, Optional, Union

# from keras import layers, backend
from keras.layers import Lambda

# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()
# BATCH, TIME, CONTEXT


# import keras.backend as K
class ngramLangModel(keras.layers.Layer):
    def __init__(self, vocab_size: int) -> None:
        # `super` call to inherit from keras.layers.Layer
        super().__init__()
        self.dropout = keras.layers.Dropout(0.1)

        self.logits = keras.layers.Dense(vocab_size, activation=None)
        self.token_embedding = keras.layers.Embedding(vocab_size, vocab_size)
        self.ff = keras.layers.Dense(vocab_size, activation=None)  # Feedforward layer

    # build used in situations where weights depend on the shape of the input tensors
    # def build(self, input_shape):
    #     if subclassers need a "state creation step"
    #     This method is used to create weights that depend on input.shape
    #     __call__ will auto build the layer if it hasn't been built yet

    # ?keras.layers.MultiHeadAttention
    # !inference mode vs training mode
    # Called in __call__, after build has been called?
    # __call__ wraps call
    # Preforms logic of applying the layer to the input tensors
    # from logits to normalized probabilities
    # CategoricalCrossentropy expects labels to be provided in a one hot rep, labels as ints use SparseCategoricalCrossentropy
    # logits = self.token_embedding(xss)
    # # print(f"logits.shape: {logits.shape}")
    # if targets is None:
    #     loss = None
    # else:
    #     B, T, C = logits.shape
    #     logits = tf.reshape(logits, (B * T, C))
    #     targets = tf.reshape(targets, (B * T))
    #     loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    #         logits=logits, labels=targets
    #     )
    #     self.add_loss(loss)
    #     return logits, loss

    # todo: Fix typo in call docs tf PR:
    def call(
        self, xss: tf.Tensor, targets: Optional[tf.Tensor] = None, training: bool = True
    ) -> Union[tf.Tensor, Tuple[tf.Tensor, tf.Tensor]]:
        xs = self.token_embedding(xss)  # embedded input
        xs = self.dropout(xs, training=training)  # dropout
        xs = self.ff(xs)  # pass through feed-forward
        logits = self.logits(xs)  # output layer

        if targets is not None:
            loss = keras.losses.sparse_categorical_crossentropy(
                targets, logits, from_logits=True
            )
            return logits, loss
        else:
            return logits
        # loss = tf.nn.softmax_cross_entropy_with_logits
        # loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    def generate(self, xss: tf.Tensor, max_new_toks: int) -> tf.Tensor:
        for _ in range(max_new_toks):
            res = self(xss)
            if isinstance(res, tuple):
                logits, loss = res
            else:
                logits = res
            logits = logits[:, -1, :]
            # obtain probability of each token (in this case char)
            # probs = tf.nn.softmax(logits)
            next_token = tf.random.categorical(logits, num_samples=1)  # (B, 1)
            # first dimension/axis is the time dimension
            xss = tf.concat([xss, next_token], axis=1)
        return xss

    # The below func can just be used as a layer. Src: https://blog.codecentric.de/move-n-gram-extraction-into-your-keras-ngram
    # def ngram_block(n, alphabet_size):
    #     def wrapped(inputs):
    #         layer = layers.Conv1D(1, n, use_bias=False, trainable=False)
    #         x = layers.Reshape((-1, 1))(inputs)
    #         x = layer(x)
    #         kernel = np.power(alphabet_size, range(0, n),
    #                           dtype=backend.floatx())
    #         layer.set_weights([kernel.reshape(n, 1, 1)])
    #         return layers.Reshape((-1,))(x)

    #     return wrapped

    def bigram(self, data):
        return tf.convert_to_tensor(
            Lambda(lambda x: [x[:, :-1] + x[:, 1:] * tp.vocab_size])(data)
        )


"""
model.eval() in torch -> training=False in tf
"""

ngram = ngramLangModel(tp.vocab_size)


# @tf.no_gradient(op_type="*")
def calculate_loss() -> dict:
    eval_iters = 400
    out = {}
    # with tf.GradientTape() as _:
    for split in ["train", "eval"]:
        losses = []
        for _ in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = ngram(X, Y)
            losses.append(loss.numpy())
        out[split] = tf.reduce_mean(losses)
    return out


def loss_fn() -> tf.Tensor:
    X, Y = get_batch("train")
    _, loss = ngram(X, Y, training=True)
    return loss


optimiser = keras.optimizers.AdamW(learning_rate=1e-3)

for timestep in range(400):
    if timestep % 100 == 0:
        loss = calculate_loss()
        print(
            f"timestep: {timestep}: training loss: {loss['train']:.4f}, eval loss: {loss['eval']:.4f}"
        )
    optimiser.minimize(loss_fn, ngram.trainable_variables)

context = tf.zeros((1, 1), dtype=tf.int64)
# print(tp.decode_mapping(ngram.generate(context, max_new_toks=60)[0].numpy().tolist()))
# ys.shape, xs.shape
# out, loss = ngram(xs, ys)
# init_xss = tf.zeros((1, 1), dtype=tf.int64)
# out = tf.nn.softmax(out)
# x = out[0].numpy().tolist()
# tp.decode_mapping(x)
# ngram.generate(init_xss, max_new_toks=10)
# print(out)
# print(loss.call(ys,out))
# x[:10]

timestep: 0: training loss: 4.5365, eval loss: 4.5365
timestep: 100: training loss: 2.8236, eval loss: 2.8186
timestep: 200: training loss: 2.6436, eval loss: 2.6506
timestep: 300: training loss: 2.5936, eval loss: 2.6039


In [47]:
print(tp.decode_mapping(ngram.generate(context, max_new_toks=100)[0].numpy().tolist()))
# train on gpu
# with tf.device('/GPU:0'):
#     model = get_model()
#     model.compile()
#     model.fit()


andicowhig thi!

ce bel—mof fuedingusanore
If locuth s be!”

I Hersis r, wfobeng amll
m ced ag wsuto
